# ICMP Pinger

Ping is a computer network application used to test whether a particular host is reachable across an IP
network. It is also used to self-test the network interface card of the computer or as a latency test. It works
by sending ICMP “echo reply” packets to the target host and listening for ICMP “echo reply” replies. The
"echo reply" is sometimes called a pong. Ping measures the round-trip time, records packet loss, and
prints a statistical summary of the echo reply packets received (the minimum, maximum, and the mean of
the round-trip times and in some versions the standard deviation of the mean).



Your task is to develop your own Ping application in Python. Your application will use ICMP but, in
order to keep it simple, will not exactly follow the official specification in RFC 1739. Note that you will
only need to write the client side of the program, as the functionality needed on the server side is built
into almost all operating systems.



You should complete the Ping application so that it sends ping requests to a specified host separated by
approximately one second. Each message contains a payload of data that includes a timestamp. After
sending each packet, the application waits up to one second to receive a reply. If one second goes by
without a reply from the server, then the client assumes that either the ping packet or the pong packet was
lost in the network (or that the server is down).

In [2]:
import os
import sys
import struct
import time
import select
from socket import *

ICMP_ECHO_REQUEST = 8


def checksum(source_string):
    """
    Calculate the checksum of a given source string.
    """
    sum = 0
    count_to = (len(source_string) // 2) * 2
    count = 0

    while count < count_to:
        this_val = source_string[count + 1] * 256 + source_string[count]
        sum = sum + this_val
        sum = sum & 0xffffffff
        count = count + 2

    if count_to < len(source_string):
        sum = sum + source_string[-1]
        sum = sum & 0xffffffff

    sum = (sum >> 16) + (sum & 0xffff)
    sum = sum + (sum >> 16)
    answer = ~sum
    answer = answer & 0xffff
    answer = answer >> 8 | (answer << 8 & 0xff00)
    return answer


def receiveOnePing(my_socket, ID, timeout, dest_addr):
    """
    Receive the ping response.
    """
    time_left = timeout

    while True:
        started_select = time.time()
        what_ready = select.select([my_socket], [], [], time_left)
        how_long_in_select = time.time() - started_select

        if what_ready[0] == []:
            return "Request timed out."

        time_received = time.time()
        rec_packet, addr = my_socket.recvfrom(1024)

        # Extract ICMP header from packet
        icmp_header = rec_packet[20:28]
        type, code, checksum, packet_ID, sequence = struct.unpack("bbHHh", icmp_header)

        if packet_ID == ID:
            bytes_sent_time = struct.unpack("d", rec_packet[28:])[0]
            return f"Reply from {dest_addr}: time={(time_received - bytes_sent_time) * 1000:.3f} ms"

        time_left = time_left - how_long_in_select
        if time_left <= 0:
            return "Request timed out."


def sendOnePing(my_socket, dest_addr, ID):
    """
    Send an ICMP echo request packet.
    """
    my_checksum = 0
    header = struct.pack("bbHHh", ICMP_ECHO_REQUEST, 0, my_checksum, ID, 1)
    data = struct.pack("d", time.time())

    # Compute the checksum
    my_checksum = checksum(header + data)

    if sys.platform == "darwin":
        my_checksum = htons(my_checksum) & 0xffff
    else:
        my_checksum = htons(my_checksum)

    header = struct.pack("bbHHh", ICMP_ECHO_REQUEST, 0, my_checksum, ID, 1)
    packet = header + data
    my_socket.sendto(packet, (dest_addr, 1))


def doOnePing(dest_addr, timeout):
    """
    Perform a single ping.
    """
    icmp = getprotobyname("icmp")
    my_socket = socket(AF_INET, SOCK_RAW, icmp)

    my_ID = os.getpid() & 0xFFFF
    sendOnePing(my_socket, dest_addr, my_ID)
    delay = receiveOnePing(my_socket, my_ID, timeout, dest_addr)

    my_socket.close()
    return delay


def ping(host, timeout=1):
    """
    Send pings continuously to a given host.
    """
    dest = gethostbyname(host)
    print(f"Pinging {dest} using Python:\n")

    while True:
        print(doOnePing(dest, timeout))
        time.sleep(1)


# Run the ping function
if __name__ == "__main__":
    ping("google.com")




Pinging 172.217.167.206 using Python:

Reply from 172.217.167.206: time=18.002 ms
Reply from 172.217.167.206: time=20.546 ms
Reply from 172.217.167.206: time=54.205 ms
Reply from 172.217.167.206: time=24.514 ms
Reply from 172.217.167.206: time=16.024 ms
Reply from 172.217.167.206: time=16.497 ms
Reply from 172.217.167.206: time=63.807 ms
Reply from 172.217.167.206: time=18.006 ms
Reply from 172.217.167.206: time=19.009 ms
Reply from 172.217.167.206: time=29.024 ms
Reply from 172.217.167.206: time=17.084 ms
Reply from 172.217.167.206: time=17.536 ms
Reply from 172.217.167.206: time=17.120 ms
Reply from 172.217.167.206: time=17.015 ms
Reply from 172.217.167.206: time=19.527 ms
Reply from 172.217.167.206: time=15.963 ms
Reply from 172.217.167.206: time=16.016 ms
Reply from 172.217.167.206: time=16.062 ms
Reply from 172.217.167.206: time=16.020 ms
Reply from 172.217.167.206: time=16.020 ms
Reply from 172.217.167.206: time=28.141 ms
Reply from 172.217.167.206: time=18.559 ms
Reply from 172.

KeyboardInterrupt: 

First, test your client by sending packets to localhost, that is, 127.0.0.1.
Then, you should see how your Pinger application communicates across the network by pinging servers
in different continents.